In [21]:
import os
os.chdir("/Users/ashwin.gangadhar/projects/mdb-innovaare-pov/")

from pymongo import MongoClient
import pandas as pd
from operators.config import MONGO_CONNECTION_URI, MONGO_COLLECTION, MONGO_DATABASE
from sentence_transformers import SentenceTransformer, util
from langchain.docstore.document import Document
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


import re
def replace_ref(text):
    text = re.sub(r'(\[[0-9]{1,2}\])', '', text)
    return text

def split_n_process_rows(text,state):
    op = []
    for ele in map(lambda x:x.strip(), text.split(".")):
        if ele!="":
            op += [(state, ele.strip() + " "+state)]
    return op

def get_emb_trnsformers():
    emb_model = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings_w = HuggingFaceEmbeddings(
        model_name=emb_model,
        cache_folder=os.getenv('SENTENCE_TRANSFORMERS_HOME')
    )
    return embeddings_w

def get_vector_store(col):
    vs = MongoDBAtlasVectorSearch(collection=col,embedding=get_emb_trnsformers(),index_name="default",embedding_key="d", text_key="doc")
    return vs

# Load directly from wiki
def query_wikipedia(title, first_paragraph_only=False):
  base_url = "https://en.wikipedia.org"
  url = f"{base_url}/w/api.php?format=json&action=query&prop=extracts&explaintext=1&titles={title}"
  if first_paragraph_only:
    url += "&exintro=1"
  data = requests.get(url).json()
  return {"state": title, "doc": list(data["query"]["pages"].values())[0]["extract"]}



In [4]:
client = MongoClient(MONGO_CONNECTION_URI)
db = client[MONGO_DATABASE]
col = db["vectest"]
sources = list(map(lambda x:query_wikipedia(x["_id"]), list(col.aggregate([{"$group":{"_id":"$state"}}]))))

In [20]:
sources[0].keys()

dict_keys(['state', 'doc'])

In [8]:

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len
)

In [14]:
texts = text_splitter.create_documents(list(map(lambda x:x["doc"], sources)))
docs = text_splitter.split_documents(texts)

In [22]:
col = db["genai"]
vec_store = get_vector_store(col)

In [23]:
vec_store.add_documents(docs)

[ObjectId('64cc7dc8ec410da5e62013d5'),
 ObjectId('64cc7dc8ec410da5e62013d6'),
 ObjectId('64cc7dc8ec410da5e62013d7'),
 ObjectId('64cc7dc8ec410da5e62013d8'),
 ObjectId('64cc7dc8ec410da5e62013d9'),
 ObjectId('64cc7dc8ec410da5e62013da'),
 ObjectId('64cc7dc8ec410da5e62013db'),
 ObjectId('64cc7dc8ec410da5e62013dc'),
 ObjectId('64cc7dc8ec410da5e62013dd'),
 ObjectId('64cc7dc8ec410da5e62013de'),
 ObjectId('64cc7dc8ec410da5e62013df'),
 ObjectId('64cc7dc8ec410da5e62013e0'),
 ObjectId('64cc7dc8ec410da5e62013e1'),
 ObjectId('64cc7dc8ec410da5e62013e2'),
 ObjectId('64cc7dc8ec410da5e62013e3'),
 ObjectId('64cc7dc8ec410da5e62013e4'),
 ObjectId('64cc7dc8ec410da5e62013e5'),
 ObjectId('64cc7dc8ec410da5e62013e6'),
 ObjectId('64cc7dc8ec410da5e62013e7'),
 ObjectId('64cc7dc8ec410da5e62013e8'),
 ObjectId('64cc7dc8ec410da5e62013e9'),
 ObjectId('64cc7dc8ec410da5e62013ea'),
 ObjectId('64cc7dc8ec410da5e62013eb'),
 ObjectId('64cc7dc8ec410da5e62013ec'),
 ObjectId('64cc7dc8ec410da5e62013ed'),
 ObjectId('64cc7dc8ec410d